<a href="https://colab.research.google.com/github/Dharshan4038/Invoice_Purchase_Order_Match/blob/main/weighted_inv_po.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence_transformers transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sentence_transformers import CrossEncoder, InputExample
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import Trainer, TrainingArguments
from torch import nn

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Invoice_PO_Match/validation_formatted 1.csv')
df.head()

,invoice_id,invoice_desc,PO_line_desc,true_mapping
0,INV-49266647,"['25103 ICE CREAM MANGO SORBETTO TRAY', 'ICE C...","['ICE CREAM MANGO SORBETTO TRAY', 'ICE CREAM G...",[{'25103 ICE CREAM MANGO SORBETTO TRAY': 'ICE ...
1,INV-49266660,"['ICE CREAM GELATO MACKINAC FDGE 3/5LT', 'ICE ...","['ICE CREAM GELATO MACKINAC FDGE', 'ICE CREAM ...",[{'ICE CREAM GELATO MACKINAC FDGE 3/5LT': 'ICE...
2,INV-49266724,['Scotch-Brite Quick Clean Griddle Liquid 700-...,"['CLEANER GRL QUICKCLEAN PCKT', 'PAD SCOUR COM...",[{'Scotch-Brite Quick Clean Griddle Liquid 700...
3,INV-49267382,"['Jersey Mikes Mini Wheat Sub', 'Jersey Mikes'...","['ROLL MINI WHEAT SUB', 'ROLL MINI WHT SUB', '...",[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WH...
4,INV-49267510,"['CASES BEEF GRND CHUCK PATTY 4oz', '14 CASES ...","['BEEF GRND CHUCK PATTY 4OZ', 'BEEF GRND THICK...",[{'CASES BEEF GRND CHUCK PATTY 4oz': 'BEEF GRN...


In [5]:
invoice = df["invoice_desc"]
purchase_order = df["PO_line_desc"]
true_mapping = df["true_mapping"]

In [6]:
import ast
for i in range(len(invoice)):
    invoice[i] = ast.literal_eval(invoice[i])
    purchase_order[i] = ast.literal_eval(purchase_order[i])
    true_mapping[i] = eval(true_mapping[i])

In [7]:
# Create a DataFrame with all possible invoice/purchase order pairs
final_data = []
for invoice_list, po_list, mapping in zip(invoice, purchase_order, true_mapping):
    mapping_dict = {list(d.keys())[0]: list(d.values())[0] for d in mapping}
    for invoice in invoice_list:
        for po in po_list:
            label = 1 if mapping_dict.get(invoice) == po else 0
            final_data.append([invoice, po, label])

In [8]:
df = pd.DataFrame(final_data, columns=['Invoice', 'Purchase Order', 'Label'])

In [9]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
model_name = "cross-encoder/stsb-roberta-base"
cross_encoder = CrossEncoder(model_name, num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# Tokenizer for the CrossEncoder model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the data for InputExample format
train_examples = [InputExample(texts=[row['Invoice'], row['Purchase Order']], label=row['Label']) for _, row in train_df.iterrows()]
val_examples = [InputExample(texts=[row['Invoice'], row['Purchase Order']], label=row['Label']) for _, row in val_df.iterrows()]

In [60]:
# Define a custom data collator
from torch.nn.utils.rnn import pad_sequence

# Custom collate function to handle padding in the DataLoader
def custom_collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]

    # Pad the input_ids and attention_mask sequences to the longest sequence in the batch
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    invoice_idxs = torch.tensor([item['invoice_idx'] for item in batch])
    po_idxs = torch.tensor([item['po_idx'] for item in batch])

    return {
        "input_ids": padded_input_ids,
        "attention_mask": padded_attention_mask,
        "invoice_idx": invoice_idxs,
        "po_idx": po_idxs
    }

In [13]:
# Define class weights (adjust for class imbalance)
class_counts = df['Label'].value_counts()
total_samples = len(df)
class_weights = [total_samples / class_counts[0], total_samples / class_counts[1]]
class_weights = torch.tensor(class_weights).to(cross_encoder.model.device)

In [14]:
class_weights

tensor([ 1.0845, 12.8367], dtype=torch.float64)

In [15]:
class WeightedLossCrossEncoder(CrossEncoder):
    def __init__(self, *args, **kwargs):
        super(WeightedLossCrossEncoder, self).__init__(*args, **kwargs)
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()
        loss = self.loss_fn(logits, labels)
        return loss, logits

In [16]:
cross_encoder = WeightedLossCrossEncoder(model_name, num_labels=1)

In [17]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01
)

# Create a Trainer instance
trainer = Trainer(
    model=cross_encoder.model,
    args=training_args,
    train_dataset=train_examples,
    eval_dataset=val_examples,
    data_collator=custom_collate_fn
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# trainer.train()

In [19]:
# output_dir = "/content/drive/MyDrive/Invoice_PO_Match/saved_model"
# cross_encoder.model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

# print(f"Model saved to {output_dir}")

In [20]:
model_name = "/content/drive/MyDrive/Invoice_PO_Match/saved_model"
cross_encoder = CrossEncoder(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
val_dataloader = DataLoader(val_examples, batch_size=8, shuffle=False, collate_fn=custom_collate_fn)

In [22]:
# Set the model to evaluation mode
cross_encoder.model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# # Make predictions on the validation set
# val_predictions = []
# val_labels = []
# val_logits = []  # Store logits for ROC-AUC calculation

# # Use a threshold to classify predictions
# threshold = 0.5

# # Evaluate predictions
# with torch.no_grad():
#     for batch in val_dataloader:
#         input_ids = batch["input_ids"].to(cross_encoder.model.device)
#         attention_mask = batch["attention_mask"].to(cross_encoder.model.device)
#         labels = batch["labels"].to(cross_encoder.model.device)

#         # Forward pass
#         outputs = cross_encoder.model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits.squeeze().cpu().numpy()

#         # Save logits for ROC-AUC
#         val_logits.extend(logits)  # Ensure logits are appended correctly

#         # Apply threshold to logits for binary classification
#         predictions = (logits >= threshold).astype(int)
#         val_predictions.extend(predictions)
#         val_labels.extend(labels.cpu().numpy())

# # Check the lengths of val_labels and val_logits
# print(f"Number of labels: {len(val_labels)}, Number of logits: {len(val_logits)}")

In [ ]:
# # Calculate metrics
# accuracy = accuracy_score(val_labels, val_predictions)
# precision = precision_score(val_labels, val_predictions)
# recall = recall_score(val_labels, val_predictions)
# f1 = f1_score(val_labels, val_predictions)
# roc_auc = roc_auc_score(val_labels, val_logits)

In [24]:
# print(f"Validation Accuracy: {accuracy:.4f}")
# print(f"Validation Precision: {precision:.4f}")
# print(f"Validation Recall: {recall:.4f}")
# print(f"Validation F1 Score: {f1:.4f}")
# print(f"Validation ROC-AUC: {roc_auc:.4f}")

In [23]:
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Confusion Matrix
# conf_matrix = confusion_matrix(val_labels, val_predictions)
# print("Confusion Matrix:")
# print(conf_matrix)

# # Optionally, plot the confusion matrix
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
# plt.ylabel('Actual')
# plt.xlabel('Predicted')
# plt.title('Confusion Matrix')
# plt.show()

In [65]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from scipy.optimize import linear_sum_assignment
import concurrent.futures

# Create a custom dataset to handle pairs of invoices and purchase orders
class InvoicePurchaseOrderDataset(Dataset):
    def __init__(self, invoices, purchase_orders, tokenizer):
        self.invoices = invoices
        self.purchase_orders = purchase_orders
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.invoices) * len(self.purchase_orders)

    def __getitem__(self, idx):
        # Calculate the correct invoice and purchase order index
        invoice_idx = idx // len(self.purchase_orders)
        po_idx = idx % len(self.purchase_orders)

        invoice = self.invoices[invoice_idx]
        purchase_order = self.purchase_orders[po_idx]

        # Tokenize the input pair
        tokenized_pair = self.tokenizer(invoice, purchase_order, padding=True, truncation=True, return_tensors="pt")

        return {
            "invoice": invoice,
            "purchase_order": purchase_order,
            "input_ids": tokenized_pair['input_ids'].squeeze(0),
            "attention_mask": tokenized_pair['attention_mask'].squeeze(0),
            "invoice_idx": invoice_idx,
            "po_idx": po_idx
        }

In [86]:
def map_invoices_to_purchase_orders(invoices, purchase_orders, cross_encoder, tokenizer, batch_size=64):
    # Initialize the dataset and DataLoader
    dataset = InvoicePurchaseOrderDataset(invoices, purchase_orders, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

    num_invoices = len(invoices)
    num_pos = len(purchase_orders)

    # Pre-allocate space for similarity matrix
    similarity_matrix = np.zeros((num_invoices, num_pos))

    # Process the dataset in parallel batches
    def process_batch(batch):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(cross_encoder.model.device)
            attention_mask = batch['attention_mask'].to(cross_encoder.model.device)
            invoice_idxs = batch['invoice_idx'].numpy()
            po_idxs = batch['po_idx'].numpy()

            # Forward pass through the model
            logits = cross_encoder.model(input_ids=input_ids, attention_mask=attention_mask)

            # Ensure that logits are squeezed to handle potential extra dimensions
            scores = logits.logits.squeeze().cpu().numpy()

            # Handle case where scores might be scalar or single values for each batch item
            if scores.ndim == 0:  # If it's a scalar, convert to array
                scores = np.array([scores])

            # Fill in the similarity matrix
            for i, (invoice_idx, po_idx) in enumerate(zip(invoice_idxs, po_idxs)):
                similarity_matrix[invoice_idx, po_idx] = scores[i]

    # Process all batches in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_batch, dataloader), total=len(dataloader), desc="Processing Batches"))

    # Use the Hungarian algorithm (linear sum assignment) for optimal one-to-one matching
    cost_matrix = -similarity_matrix  # Convert to cost matrix for minimization
    row_idx, col_idx = linear_sum_assignment(cost_matrix)

    # Create the final result with one-to-one mappings
    results = [{invoices[i]: purchase_orders[j]} for i, j in zip(row_idx, col_idx)]

    return results

In [93]:
invoices = df["invoice_desc"][21]
purchase_orders = df["PO_line_desc"][21]

In [94]:
print(len(invoices))
print(len(purchase_orders))

3
3


In [95]:
mapped_results = map_invoices_to_purchase_orders(invoices, purchase_orders, cross_encoder, tokenizer)

# Output the results
print("Mapped Invoices to Purchase Orders:")
for mapping in mapped_results:
    print(mapping)

Processing Batches: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]

Mapped Invoices to Purchase Orders:
{'Fritter Bread - Blueberry': 'BREAD BLUEBERRY FRTTR'}
{'Fritter Bread - Apple': 'BREAD APPLE FRTTR'}
{'Fritter Bread - Raspberry': 'BREAD RASPBERRY FRTTR'}


In [96]:
import pandas as pd
import ast
import torch

# Load the original CSV containing the invoice and purchase order data
df = pd.read_csv('/content/drive/MyDrive/Invoice_PO_Match/validation_formatted 1.csv')

# Extract the necessary columns: 'invoice_id', 'invoice_desc', 'PO_line_desc', 'true_mapping'
df = df[['invoice_id', 'invoice_desc', 'PO_line_desc', 'true_mapping']]

# Parse the 'invoice_desc', 'PO_line_desc', and 'true_mapping' columns from strings to lists/dictionaries
for i in range(len(df)):
    df.at[i, 'invoice_desc'] = ast.literal_eval(df.at[i, 'invoice_desc'])
    df.at[i, 'PO_line_desc'] = ast.literal_eval(df.at[i, 'PO_line_desc'])
    df.at[i, 'true_mapping'] = ast.literal_eval(df.at[i, 'true_mapping'])

In [97]:
# Function to compute and add the 'model_output' for each row
def compute_model_output(df, cross_encoder, tokenizer):
    model_output = []

    for i, row in df.iterrows():
        invoices = row['invoice_desc']
        purchase_orders = row['PO_line_desc']
        print(i)
        # Use the map_invoices_to_purchase_orders function to get the mapping
        mapping_result = map_invoices_to_purchase_orders(invoices, purchase_orders, cross_encoder, tokenizer)
        print(mapping_result)
        # Append the mapping result to the list
        model_output.append(mapping_result)

    # Add the 'model_output' column to the dataframe
    df['model_output'] = model_output

    return df

In [98]:
c = 0
for i in range(946):
  if len(df["invoice_desc"][i]) == len(df["PO_line_desc"][i]):
    c += 1
print(c)

946


In [ ]:
# Compute the model output for the dataframe
df_with_model_output = compute_model_output(df, cross_encoder, tokenizer)

# Save the dataframe to a CSV file
output_csv_path = '/content/drive/MyDrive/Invoice_PO_Match/mapped_invoices_po_output.csv'
df_with_model_output.to_csv(output_csv_path, index=False)

print(f"Model output saved to: {output_csv_path}")

In [100]:
fin_df = pd.read_csv("/content/drive/MyDrive/Invoice_PO_Match/mapped_invoices_po_output.csv")
fin_df.head()

,invoice_id,invoice_desc,PO_line_desc,true_mapping,model_output
0,INV-49266647,"['25103 ICE CREAM MANGO SORBETTO TRAY', 'ICE C...","['ICE CREAM MANGO SORBETTO TRAY', 'ICE CREAM G...",[{'25103 ICE CREAM MANGO SORBETTO TRAY': 'ICE ...,[{'25103 ICE CREAM MANGO SORBETTO TRAY': 'ICE ...
1,INV-49266660,"['ICE CREAM GELATO MACKINAC FDGE 3/5LT', 'ICE ...","['ICE CREAM GELATO MACKINAC FDGE', 'ICE CREAM ...",[{'ICE CREAM GELATO MACKINAC FDGE 3/5LT': 'ICE...,[{'ICE CREAM GELATO MACKINAC FDGE 3/5LT': 'ICE...
2,INV-49266724,['Scotch-Brite Quick Clean Griddle Liquid 700-...,"['CLEANER GRL QUICKCLEAN PCKT', 'PAD SCOUR COM...",[{'Scotch-Brite Quick Clean Griddle Liquid 700...,[{'Scotch-Brite Quick Clean Griddle Liquid 700...
3,INV-49267382,"['Jersey Mikes Mini Wheat Sub', 'Jersey Mikes'...","['ROLL MINI WHEAT SUB', 'ROLL MINI WHT SUB', '...",[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WH...,[{'Jersey Mikes Mini Wheat Sub': 'ROLL MINI WH...
4,INV-49267510,"['CASES BEEF GRND CHUCK PATTY 4oz', '14 CASES ...","['BEEF GRND CHUCK PATTY 4OZ', 'BEEF GRND THICK...",[{'CASES BEEF GRND CHUCK PATTY 4oz': 'BEEF GRN...,[{'CASES BEEF GRND CHUCK PATTY 4oz': 'BEEF GRN...


In [124]:
df["model_output"][74]

[{'BEEF': 'BEEF TACO FLLNG RED PPR'},
 {'BEEF': 'BEEF PATTY 3-1 BEST EVER'},
 {'PATTIE': 'BEEF PIZZA PATTIES 3.5OZ'},
 {'BRD': 'BEEF BURGER TNDRBT 81/19 3-1'},
 {'BEEF': 'BEEF GRND PTY 81\\19 SILVER IQF'},
 {'BEEF': 'BEEF PATTY SEASONED BIG BITE'},
 {'MEATBALL': 'MEATBALL BEEF PRCKD 1 OZ'},
 {'BEEF': 'BEEF ROAST POT CKD FRZ'},
 {'STEAK': 'STEAK SIRL RESD FRZN'},
 {'BEEF': 'CHICKEN TENDER BRD FRITT DIXIE'},
 {'BRD': 'CHICKEN BNLS BRD WINGS'}]

In [125]:
df["true_mapping"][74]

[{'BEEF': 'BEEF PATTY SEASONED BIG BITE'},
 {'BEEF': 'BEEF PATTY 3-1 BEST EVER'},
 {'PATTIE': 'BEEF PIZZA PATTIES 3.5OZ'},
 {'BRD': 'CHICKEN TENDER BRD FRITT DIXIE'},
 {'BEEF': 'BEEF BURGER TNDRBT 81/19 3-1'},
 {'BEEF': 'BEEF GRND PTY 81\\19 SILVER IQF'},
 {'MEATBALL': 'MEATBALL BEEF PRCKD 1 OZ'},
 {'BEEF': 'BEEF ROAST POT CKD FRZ'},
 {'STEAK': 'STEAK SIRL RESD FRZN'},
 {'BEEF': 'BEEF TACO FLLNG RED PPR'},
 {'BRD': 'CHICKEN BNLS BRD WINGS'}]

In [117]:
def compare_lists_of_dicts(list1, list2):
    # Convert each dict to a sorted tuple of key-value pairs
    list1_normalized = sorted([sorted(d.items()) for d in list1])
    list2_normalized = sorted([sorted(d.items()) for d in list2])

    # Compare the normalized lists
    return list1_normalized == list2_normalized

In [121]:
compare_lists_of_dicts(df["model_output"][30],df["true_mapping"][30])

False

In [127]:
correct = 0
for i in range(len(df)):
  if compare_lists_of_dicts(df["model_output"][i],df["true_mapping"][i]):
    correct = correct + 1
  else:
    print(i)
acc = correct/len(df)

30
74
75
94
106
130
156
164
244
288
317
422
521
549
598
605
608
623
627
667
756
766
798
802
834
843
861
894
932


In [128]:
acc = acc*100
acc

96.93446088794926